# Data Staging for GDP Data

## Importing Libraries and Reading the Data

In [198]:
import pandas as pd
import numpy as np

In [199]:
df = pd.read_csv('data/raw_gdp.csv')

## Previewing the Dataset

In [200]:
df.head()

,REF_DATE,GEO,DGUID,North American Industry Classification System (NAICS),UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1997,Canada,2016A000011124,All industries [T001],Dollars,81,millions,6,v64605602,1.1,840500,NaN,NaN,NaN,0
1,1997,Canada,2016A000011124,Goods-producing industries [T002],Dollars,81,millions,6,v64605713,1.2,269364,NaN,NaN,NaN,0
2,1997,Canada,2016A000011124,Service-producing industries [T003],Dollars,81,millions,6,v64605824,1.3,571136,NaN,NaN,NaN,0
3,1997,Canada,2016A000011124,Industrial production [T010],Dollars,81,millions,6,v64605841,1.4,205651,NaN,NaN,NaN,0
4,1997,Canada,2016A000011124,Non-durable manufacturing industries [T011],Dollars,81,millions,6,v64605852,1.5,60046,NaN,NaN,NaN,0


## Unique Values in Each Column

In [201]:
unique_values = df[['GEO', 'DGUID', 'UOM', 'UOM_ID','SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR' , 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']].apply(lambda x: x.unique())
unique_values

GEO                                                       [Canada]
DGUID                                             [2016A000011124]
UOM                                                      [Dollars]
UOM_ID                                                        [81]
SCALAR_FACTOR                                           [millions]
SCALAR_ID                                                      [6]
VECTOR           [v64605602, v64605713, v64605824, v64605841, v...
COORDINATE       [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, ...
VALUE            [840500, 269364, 571136, 205651, 60046, 85467,...
STATUS                                                       [nan]
SYMBOL                                                       [nan]
TERMINATED                                                   [nan]
DECIMALS                                                       [0]
dtype: object

## Dropping Unnecessary Columns

In [202]:
df = df.drop(columns=['GEO', 'DGUID', 'UOM', 'UOM_ID', 'SCALAR_FACTOR',  'SCALAR_ID', 'VECTOR', 'COORDINATE', 'STATUS', 'SYMBOL', 'SYMBOL', 'TERMINATED', 'DECIMALS'])

## Update value using the scalar factor (millions)

In [203]:
df['VALUE'] = df['VALUE'].apply(lambda x: x*1000000)

In [204]:
df.head()

,REF_DATE,North American Industry Classification System (NAICS),VALUE
0,1997,All industries [T001],840500000000
1,1997,Goods-producing industries [T002],269364000000
2,1997,Service-producing industries [T003],571136000000
3,1997,Industrial production [T010],205651000000
4,1997,Non-durable manufacturing industries [T011],60046000000


## Examining Data Types

In [205]:
df.dtypes

REF_DATE                                                  int64
North American Industry Classification System (NAICS)    object
VALUE                                                     int64
dtype: object

## Renaming Columns for Clarity and Simplicity

In [206]:
df = df.rename(columns={'REF_DATE': 'Date', 'North American Industry Classification System (NAICS)': 'NAICS', 'VALUE': 'Value'})

In [207]:
df.head()

,Date,NAICS,Value
0,1997,All industries [T001],840500000000
1,1997,Goods-producing industries [T002],269364000000
2,1997,Service-producing industries [T003],571136000000
3,1997,Industrial production [T010],205651000000
4,1997,Non-durable manufacturing industries [T011],60046000000


## Checking for Missing Values

In [208]:
df['Date'].isnull().unique()

array([False])

In [209]:
df['NAICS'].isnull().unique()

array([False])

In [210]:
df['Value'].isnull().unique()

array([False])

## Filtering Date to Start from 2000

In [211]:
df = df[df['Date'] >= 2000]

In [212]:
df.head()

,Date,NAICS,Value
1008,2000,All industries [T001],1027444000000
1009,2000,Goods-producing industries [T002],347862000000
1010,2000,Service-producing industries [T003],679582000000
1011,2000,Industrial production [T010],275215000000
1012,2000,Non-durable manufacturing industries [T011],72315000000


## Reset indeces

In [213]:
df.reset_index(drop=True, inplace=True)

In [214]:
df

,Date,NAICS,Value
0,2000,All industries [T001],1027444000000
1,2000,Goods-producing industries [T002],347862000000
2,2000,Service-producing industries [T003],679582000000
3,2000,Industrial production [T010],275215000000
4,2000,Non-durable manufacturing industries [T011],72315000000
...,...,...,...
7054,2020,Federal government public administration (exce...,42564000000
7055,2020,Provincial and territorial public administrati...,40664000000
7056,2020,"Local, municipal, regional and aboriginal publ...",60291000000
7057,2020,"Local, municipal and regional public administr...",54541000000


## Check if NAICS are all possible sectors

In [215]:
len(df['NAICS'].unique())

337

In [216]:
unique_NAICS = df['NAICS'].unique()

In [217]:
unique_NAICS

array(['All industries [T001]', 'Goods-producing industries [T002]',
       'Service-producing industries [T003]',
       'Industrial production [T010]',
       'Non-durable manufacturing industries [T011]',
       'Durable manufacturing industries [T012]',
       'Information and communication technology sector [T013]',
       'Information and communication technology, manufacturing [T014]',
       'Information and communication technology, services [T015]',
       'Energy sector [T016]', 'Public sector [T018]',
       'All industries (except cannabis sector) [T020]',
       'Cannabis sector [T021]', 'Cannabis sector (licensed) [T022]',
       'Cannabis sector (unlicensed) [T023]',
       'All industries (except unlicensed cannabis sector) [T024]',
       'Agriculture, forestry, fishing and hunting [11]',
       'Crop and animal production [11A]', 'Crop production [111]',
       'Crop production (except cannabis) [111X]',
       'Greenhouse, nursery and floriculture production (except

In [218]:
len(unique_NAICS)

337

## Function to drop any subsectors (or total of sectors)

In [219]:
naics_sector_codes = [
    "[11]", "[21]", "[22]", "[23]", "[31-33]", "[42]", "[44-45]", "[48-49]", "[51]",
    "[52]", "[53]", "[54]", "[55]", "[56]", "[61]", "[62]", "[71]", "[72]", "[81]", "[91]"
]
def is_sector(naics):
    return naics.split()[-1] in naics_sector_codes

## Create an updated df that only contains NAICS sectors (with codes)

In [220]:
updated_df = df.drop(df[~df['NAICS'].apply(is_sector)].index)

In [221]:
updated_df = updated_df.reset_index(drop=True)

In [222]:
updated_df

,Date,NAICS,Value
0,2000,"Agriculture, forestry, fishing and hunting [11]",23120000000
1,2000,"Mining, quarrying, and oil and gas extraction ...",58615000000
2,2000,Utilities [22],26770000000
3,2000,Construction [23],51054000000
4,2000,Manufacturing [31-33],188303000000
...,...,...,...
394,2020,Health care and social assistance [62],168768000000
395,2020,"Arts, entertainment and recreation [71]",11842000000
396,2020,Accommodation and food services [72],31073000000
397,2020,Other services (except public administration) ...,39930000000


In [223]:
updated_df['NAICS'].unique()

array(['Agriculture, forestry, fishing and hunting [11]',
       'Mining, quarrying, and oil and gas extraction [21]',
       'Utilities [22]', 'Construction [23]', 'Manufacturing [31-33]',
       'Retail trade [44-45]', 'Transportation and warehousing [48-49]',
       'Information and cultural industries [51]',
       'Finance and insurance [52]',
       'Real estate and rental and leasing [53]',
       'Professional, scientific and technical services [54]',
       'Management of companies and enterprises [55]',
       'Administrative and support, waste management and remediation services [56]',
       'Educational services [61]',
       'Health care and social assistance [62]',
       'Arts, entertainment and recreation [71]',
       'Accommodation and food services [72]',
       'Other services (except public administration) [81]',
       'Public administration [91]'], dtype=object)

## Check if there are any naics duplicates

In [224]:
duplicated = updated_df.duplicated(subset=['Date', 'NAICS'], keep=False)
duplicated.any()

False

## Map NAICS to employment NAICS (Merge NAICS with codes 51, 71 together as well as 55, 56)

In [225]:
#Merge NAICS [51], [71] along with [55], [56] and rename NAICS sector
updated_df['NAICS'].replace({'Arts, entertainment and recreation [71]': 'Information and cultural industries [51]'}, inplace=True)

updated_df['NAICS'].replace({'Administrative and support, waste management and remediation services [56]': 'Management of companies and enterprises [55]'}, inplace=True)


df_grouped = updated_df.groupby(['Date', 'NAICS'], as_index=False)['Value'].sum()

df_grouped.loc[df_grouped['NAICS'] == 'Information and cultural industries [51]', 'NAICS'] = 'Information, culture and recreation [51, 71]'
df_grouped.loc[df_grouped['NAICS'] == 'Management of companies and enterprises [55]', 'NAICS'] = 'Business, building and other support services [55, 56]'


In [226]:
df_grouped

,Date,NAICS,Value
0,2000,Accommodation and food services [72],22863000000
1,2000,"Agriculture, forestry, fishing and hunting [11]",23120000000
2,2000,Construction [23],51054000000
3,2000,Educational services [61],51839000000
4,2000,Finance and insurance [52],62648000000
...,...,...,...
352,2020,Public administration [91],158772000000
353,2020,Real estate and rental and leasing [53],293886000000
354,2020,Retail trade [44-45],106641000000
355,2020,Transportation and warehousing [48-49],95339000000


In [227]:
df_grouped['Value'].min()

21052000000

## Check if any values are 0

In [228]:
df_grouped[df_grouped['Value'] == 0]

,Date,NAICS,Value


# Binning the Dataframe

In [229]:
bin_edges = pd.cut(df_grouped['Value'], bins=10, retbins=True)[1]
bin_edges = np.ceil(bin_edges / 1e9) * 1e9
df_grouped['Value Binned'] = pd.cut(df_grouped['Value'], bins=bin_edges)
df_grouped

,Date,NAICS,Value,Value Binned
0,2000,Accommodation and food services [72],22863000000,"(21000000000.0, 49000000000.0]"
1,2000,"Agriculture, forestry, fishing and hunting [11]",23120000000,"(21000000000.0, 49000000000.0]"
2,2000,Construction [23],51054000000,"(49000000000.0, 76000000000.0]"
3,2000,Educational services [61],51839000000,"(49000000000.0, 76000000000.0]"
4,2000,Finance and insurance [52],62648000000,"(49000000000.0, 76000000000.0]"
...,...,...,...,...
352,2020,Public administration [91],158772000000,"(158000000000.0, 185000000000.0]"
353,2020,Real estate and rental and leasing [53],293886000000,"(267000000000.0, 294000000000.0]"
354,2020,Retail trade [44-45],106641000000,"(103000000000.0, 131000000000.0]"
355,2020,Transportation and warehousing [48-49],95339000000,"(76000000000.0, 103000000000.0]"


## Export the updated dataframe to CSV

In [230]:
df_grouped.to_csv('data/cleaned_gdp.csv', index=False)